# Aplikacja internetowa do przewidywania czy pacjent może doznać udaru

### Opis

Według Światowej Organizacji Zdrowia (WHO) udar jest drugą najczęstszą przyczyną śmierci na świecie i odpowiada za około 11% wszystkich zgonów.

Ta aplikacja służy do przewidywania, czy pacjent może doznać udaru na podstawie parametrów wejściowych, takich jak płeć, wiek, wybrane choroby i palenie tytoniu.

### Cel

Celem projektu było zidentyfikowanie czynników ryzyka wpływających na ryzyko udaru.

### Wyniki

Zaobserwowano, że istotny wpływ na ryzyko udaru mają choroby takie jak nadciśnienie lub choroby serca. Ponad to zauważalnie większe ryzyko udaru występuje wśród osób samozatrudnionych.

### Dane
Dane pochodzą ze datasetu [Stroke Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)





## Import potrzebnych bibliotek

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.exceptions import NotFittedError

## Wczytanie danych



In [10]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/stroke/healthcare-dataset-stroke-data.csv'
data = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Przygotowanie danych

In [11]:
data = data.drop('id', axis=1).drop('avg_glucose_level', axis=1)
data = data.dropna()
X = data.drop('stroke', axis=1)
y = data['stroke']

categorical_features = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
transformer = ColumnTransformer([('encoder', OneHotEncoder(), categorical_features)], remainder='passthrough')
X = transformer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Eksploracyjna Analiza Danych

In [12]:
print("Shape")
print(data.shape)
print("First 10")
print(data.head(10))
print("Types")
print(data.dtypes)
print("Description")
print(data.describe())
print("Nulls")
print(data.isnull().sum())

Shape
(4909, 10)
First 10
    gender   age  hypertension  heart_disease ever_married      work_type  \
0     Male  67.0             0              1          Yes        Private   
2     Male  80.0             0              1          Yes        Private   
3   Female  49.0             0              0          Yes        Private   
4   Female  79.0             1              0          Yes  Self-employed   
5     Male  81.0             0              0          Yes        Private   
6     Male  74.0             1              1          Yes        Private   
7   Female  69.0             0              0           No        Private   
9   Female  78.0             0              0          Yes        Private   
10  Female  81.0             1              0          Yes        Private   
11  Female  61.0             0              1          Yes       Govt_job   

   Residence_type   bmi   smoking_status  stroke  
0           Urban  36.6  formerly smoked       1  
2           Rural  32.5 

## Trenowanie modelu

In [13]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

## Korelacja danych

In [14]:
numeric_data = data.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numeric_data.corr()
print(correlation_matrix)

                    age  hypertension  heart_disease       bmi    stroke
age            1.000000      0.274425       0.257123  0.333398  0.232331
hypertension   0.274425      1.000000       0.115991  0.167811  0.142515
heart_disease  0.257123      0.115991       1.000000  0.041357  0.137938
bmi            0.333398      0.167811       0.041357  1.000000  0.042374
stroke         0.232331      0.142515       0.137938  0.042374  1.000000


## Przykładowa predykcja na nowych danych

In [15]:
new_data = pd.DataFrame([['Female', 23, 0, 0, 'No', 'Private', 'Rural', 17.2, 'smokes']],
                        columns=['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'bmi', 'smoking_status'])

encoded = transformer.transform(new_data)
predicted = model.predict(encoded)
print(f"Predicted: {'Stroke' if predicted.round() >= 1 else 'No stroke'}")

Predicted: No stroke
